<h1><center>CMPE 462 - Quiz 4&5 <br>Implementing a Regression Tree<br>Due: May 3, 2020, 23:59</center></h1>

# Introduction

Decision trees can learn rules to map features to both continous and discrete outputs. In this quiz, you will consider the continous case and implement a regression tree to predict house prices in Boston. You will also conduct small data analysis and evaluation procedures. This notebook will guide you through.

## (10 pts) Task 1: Dataset 

Run the cells below to load Boston house prices dataset using `scikit-learn`. You can find more detail on the dataset [here](https://scikit-learn.org/stable/datasets/index.html#boston-dataset).

When the dataset is loaded, construct train and test matrices, by allocating **first 400 samples** to train and the rest to test. ___Do not shuffle or randomly sample the feature matrix___.

In [ ]:
import numpy as np
from sklearn.datasets import load_boston
import seaborn as sns

In [ ]:
boston = load_boston()
X_y = np.column_stack([boston['data'], boston['target']])
np.random.seed(1)
np.random.shuffle(X_y)
X, y = X_y[:,:-1], X_y[:,-1]
X_train, y_train = X[:400], y[:400]
X_test, y_test = X[400:], y[400:]
del X, y, X_y

In [ ]:
print(boston.keys()) # Keys of bunch
print(boston.feature_names)

In [ ]:
print("Train Data:", "X:", X_train.shape, "y:", y_train.shape)
print("Test Data:", "X:", X_test.shape, "y:", y_test.shape) 

**(10 pts)** Unlike the dataset in Project 1, Boston dataset is high-dimensional and we cannot observe the relations between all features and the output with a single scatter plot. What we can do is to visualize the correlations between both features and house prices using a heatmap. So, stack the features and the outputs in a single matrix and compute correlation between all pairs of columns. Visualize the correlation coefficient matrix as a heatmap, which is $(N+1)x(N+1)$, where $N$ is the number of features in Boston dataset. You can check out `corrcoef` and `heatmap` functions from `numpy` and `seaborn` libraries, respectively. You can use diverging color palette to emphasize both positive and negative correlations.

Do you observe strong correlations between any pair of features or certain features and house price? 

In [ ]:
concat_mat = np.concatenate([X_train.transpose(), y_train.reshape(1,-1)])
corr = np.corrcoef(concat_mat.reshape(-1,400))
names = list(boston.feature_names) + ["y"] 
sns.heatmap(corr, xticklabels=names, yticklabels=names)

#### First of all, all features and target have maximum correlation with itself, as expected. In addition, according to the heatmap above, target has high correlation with the feature "LSTAT". "RAD" and "TAX" also have a very strong correlation. For other features, strong correlations can be obtained as in the below cell.

In [ ]:
print("Strong Correlations")
for i in range(len(names)):
    second_largest_i = np.argsort(abs(corr)[i,:])[-2]
    print(names[i], "has the strongest correlation with", names[second_largest_i]+ "("+str(second_largest_i)+")", " - Corr:", corr[i,:][second_largest_i])

print("\nWeak Correlations")
for i in range(len(names)):
    second_largest_i = np.argsort(abs(corr)[i,:])[0]
    print(names[i], "has the weakest correlation with", names[second_largest_i]+ "("+str(second_largest_i)+")", " - Corr:", corr[i,:][second_largest_i])


## (80 pts) Task 2: Regression Tree


**(15 pts)** Let us now build the regression tree. We start by implementing the splitting criteria which is used to decide the most discriminative features at each step. We stick to lecture notes and use $RSS$ which is computed as follows:

$$RSS =\sum_{\text {left }}\left(y_{i}-y_{L}^{*}\right)^{2}+\sum_{\text {right }}\left(y_{i}-y_{R}^{*}\right)^{2}$$

where $y_L^* and y_L^*$ are mean y-value of left and right nodes.

When you implement $RSS$, pick the most correlated and least correlated feature with the output prices according to previous step. Note that correlation can be both positive and negative! For both features, compute $RSS$ for every possible split threshold and plot thresholds versus RSS scores.

Do two features display different characteristics?

#### y has the strongest correlation with LSTAT(12)  - Corr: -0.7353565653971893
#### y has the weakest correlation with CHAS(3)  - Corr: 0.1964473680907214

In [ ]:
def rss(X_i, y, threshold):
    y_left = y[X_i<threshold]
    y_right = y[X_i>=threshold]
    y_left_mean = np.mean(y_left)
    y_right_mean = np.mean(y_right)
    return np.sum(np.power((y_left - y_left_mean),2)) + np.sum(np.power((y_right - y_right_mean),2)) 

def find_best_threshold(X_i, y):
    thresholds = []
    rss_values = []
    sorted_X_i = sorted(X_i)
    for i in range(len(X_i)-1):
        split_threshold = (sorted_X_i[i]+sorted_X_i[i+1])/2
        current_rss = rss(X_i, y, split_threshold)
        thresholds.append(split_threshold)
        rss_values.append(current_rss)
    
    return thresholds[np.argmin(rss_values)], thresholds, rss_values 

In [ ]:
LSTAT = X_train[:,12]
CHAS = X_train[:,3]

LSTAT_threshold, LSTAT_thresholds, LSTAT_rss_values = find_best_threshold(LSTAT, y_train)
CHAS_threshold, CHAS_thresholds, CHAS_rss_values = find_best_threshold(CHAS, y_train)

In [ ]:
print("Best threshold for LSTAT:", LSTAT_threshold, "RSS:")
sns.scatterplot(LSTAT_thresholds, LSTAT_rss_values)

In [ ]:
print("Best threshold for CHAS:", CHAS_threshold)
sns.scatterplot(CHAS_thresholds, CHAS_rss_values)

#### Best threshold for LSTAT gives a better rss value than the one for CHAS (~18k < ~30k). That means, LSTAT gives more information when used as decision feature with selected threshold as expected, because it has stronger correlation with the target. In addition, we can say that CHAS is a binary variable, because there are only 3 threshold that we can choose which are 0, 0.5 and 1, as we can see from the graph above.

**(45 pts)** Now implement the training procedure of a regression tree using $RSS$ as split criteria. Build a rule tree recursively by traversing all features and considering each split threshold to find the optimum split, at every node.

You are free to implement training procedure as a standalone function or part of a class, but in any case use maximum depth as the stopping condition.

In [ ]:
class RegressionTree:
    def __init__(self, depth=0):
        self.depth = depth
        self.feature_index = None
        self.threshold = None
        self.prediction = None # If not a leaf, this is None.
        self.left_node = None # Another RegressionTree
        self.right_node = None # Another RegressionTree


def fit_tree(tree, X, y, max_depth, prev_mean=None):

    if max_depth < 1:
        AssertionError("max_depth must be higher than 0")

    if tree.depth == max_depth or len(y) == 1:
        tree.prediction = prev_mean
        return

    min_rss = None
    best_feature_i = None
    best_threshold = None

    for feature_i in range(X.shape[1]):
        X_i = X[:,feature_i]
        threshold, _, rss_values = find_best_threshold(X_i, y)
        if min_rss is None or min_rss > np.min(rss_values):
            min_rss = np.min(rss_values)
            best_feature_i = feature_i
            best_threshold = threshold
    
    tree.feature_index = best_feature_i
    tree.threshold = best_threshold
    tree.left_node = RegressionTree(tree.depth+1)
    tree.right_node = RegressionTree(tree.depth+1)

    left_X = X[X[:, tree.feature_index] < tree.threshold] 
    left_y = y[X[:, tree.feature_index] < tree.threshold]
    left_prev_mean = np.mean(left_y)

    right_X = X[X[:, tree.feature_index] >= tree.threshold] 
    right_y = y[X[:, tree.feature_index] >= tree.threshold]
    right_prev_mean = np.mean(right_y)

    fit_tree(tree.left_node, left_X, left_y, max_depth, left_prev_mean)
    fit_tree(tree.right_node, right_X, right_y, max_depth, right_prev_mean)



**(15 pts)** Having implemented the regression tree, now write a procedure to predict test features. Given a sample, this procedure should follow the rules learned during the training to arrive at a leaf and predict the output as the mean output of the arrived leaf samples. 

If you have implemented a regression tree class, you can insert this procedure as a class function 

In [ ]:
def predict_tree(tree, X):
    # X should have the shape of (N,) where N is the number of features.

    if tree.prediction:
        return tree.prediction

    if X[tree.feature_index] < tree.threshold:
        return predict_tree(tree.left_node, X)
    else:
        return predict_tree(tree.right_node, X) 
    

**(5 pts)** Train your model using a max depth of 3 and visualize the resulting tree. You can use an external tool such as draw.io or LaTeX for drawing. Annotate the nodes with split columns and thresholds. You can view the tree in this [link](https://scikit-learn.org/stable/modules/tree.html#tree) as an example. 

In [ ]:
tree = RegressionTree()
fit_tree(tree, X_train, y_train, max_depth=3) 

In [ ]:
print("1st node:", names[tree.feature_index], tree.threshold, tree.prediction) 

print("2nd node:", names[tree.left_node.feature_index], tree.left_node.threshold, tree.left_node.prediction) 
print("3rd node:", names[tree.right_node.feature_index], tree.right_node.threshold, tree.right_node.prediction) 

print("4th node:", names[tree.left_node.left_node.feature_index], tree.left_node.left_node.threshold, tree.left_node.left_node.prediction) 
print("5th node:", names[tree.left_node.right_node.feature_index], tree.left_node.right_node.threshold, tree.left_node.right_node.prediction) 
print("6th node:", names[tree.right_node.left_node.feature_index], tree.right_node.left_node.threshold, tree.right_node.left_node.prediction) 
print("7th node:", names[tree.right_node.right_node.feature_index], tree.right_node.right_node.threshold, tree.right_node.right_node.prediction)

print("8th node:", tree.left_node.left_node.left_node.feature_index, tree.left_node.left_node.left_node.threshold, tree.left_node.left_node.left_node.prediction) 
print("9th node:", tree.left_node.left_node.right_node.feature_index, tree.left_node.left_node.right_node.threshold, tree.left_node.left_node.right_node.prediction) 
print("10th node:", tree.left_node.right_node.left_node.feature_index, tree.left_node.right_node.left_node.threshold, tree.left_node.right_node.left_node.prediction) 
print("11th node:", tree.left_node.right_node.right_node.feature_index, tree.left_node.right_node.right_node.threshold, tree.left_node.right_node.right_node.prediction)
print("12th node:", tree.right_node.left_node.left_node.feature_index, tree.right_node.left_node.left_node.threshold, tree.right_node.left_node.left_node.prediction) 
print("13th node:", tree.right_node.left_node.right_node.feature_index, tree.right_node.left_node.right_node.threshold, tree.right_node.left_node.right_node.prediction) 
print("14th node:", tree.right_node.right_node.left_node.feature_index, tree.right_node.right_node.left_node.threshold, tree.right_node.right_node.left_node.prediction) 
print("15th node:", tree.right_node.right_node.right_node.feature_index, tree.right_node.right_node.right_node.threshold, tree.right_node.right_node.right_node.prediction) 



#### As we can see from the image, RM and LSTAT were used as desicion at the top of the tree. It is expected, because they have a good correlation with y.
<img src="tree.png">

## **(10 pts)** Task 3: Evaluation

**(5 pts)** Now is time to pick the best maximum depth and observe your tree's performance! Implement a 5-fold cross validation procedure to experiment with maximum depths from 3 to 10. Report mean and standard deviation for each depth and pick the best one. For comparison you can use $R^2$, which is a metric frequently used to evaluate regression models. You can use `r2_score` function of `scikit-learn` and read more [here](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score).

In [ ]:
from sklearn.metrics import r2_score

def get_r2_score(tree, X, y):
    preds = []
    for i in range(len(X)):
        preds.append(predict_tree(tree, X[i]))
    return r2_score(y, preds)        

In [ ]:
depths = list(range(3,11))
scores = []

for depth_i, max_depth in enumerate(depths):
    
    tree = RegressionTree()
    split_lenght = len(X_train)//5 
    split_indices = [split_lenght*i for i in range(5)]
    scores.append([])
    
    for i in range(5):
        
        val_indices = np.zeros(X_train.shape[0], dtype="bool")
        val_indices[split_indices[i]:split_indices[i]+split_lenght] = True

        X_val_split = X_train[val_indices,:]
        X_train_split = X_train[np.logical_not(val_indices),:]
        y_val_split = y_train[val_indices]
        y_train_split = y_train[np.logical_not(val_indices)]

        fit_tree(tree, X_train_split, y_train_split, max_depth)
        scores[depth_i].append(get_r2_score(tree, X_val_split, y_val_split))
    
    print("Max Depth: {} - Mean R2={}, Std R2={}".format(max_depth, np.mean(scores[depth_i]), np.sqrt(np.var(scores[depth_i]))))


**(5 pts)** To conclude, train your tree one last time on the whole training data with the depth you picked in the previous section. Generate predictions on both training and test sets and report $R^2$ scores.

I chose 6 as max depth, since it gives the highest mean R2 score.

In [ ]:
tree = RegressionTree()
fit_tree(tree, X_train, y_train, max_depth=6)

print("R2 score for Train Set:", get_r2_score(tree, X_train, y_train))

print("R2 score for Test Set", get_r2_score(tree, X_test, y_test))